In [14]:
from datetime import datetime
import enum
from typing import Dict, Any
from sqlalchemy import (
    create_engine,
    Column,
    Integer,
    String,
    Float,
    DateTime,
    Enum,
    ForeignKey
)
from sqlalchemy.orm import (
    sessionmaker,
    declarative_base,
    relationship,
    scoped_session
)

Base = declarative_base()

class TransactionType(enum.Enum):
    BUY = "BUY"
    SELL = "SELL"


class Trader(Base):
    """
    A Trader ORM model that keeps track of balance and has relationships to
    transaction records. Provides methods for buying/selling stocks and
    computing performance statistics.
    """
    __tablename__ = "traders"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)
    balance = Column(Float, nullable=False, default=0.0)

    transactions = relationship("Transaction", back_populates="trader", cascade="all, delete-orphan")

    def buy_stock(self, session, symbol: str, quantity: int, price: float):
        """
        Attempt to buy `quantity` shares of `symbol` at `price`.
        Deducts cost from the trader's balance if sufficient funds are available.
        Raises ValueError if not enough balance.
        """
        total_cost = price * quantity
        if self.balance < total_cost:
            raise ValueError("Insufficient balance to execute buy.")
        
        # Deduct balance
        self.balance -= total_cost
        
        # Create a transaction record
        tx = Transaction(
            trader=self,
            symbol=symbol,
            quantity=quantity,
            price=price,
            transaction_type=TransactionType.BUY
        )
        session.add(tx)
        session.commit()

    def sell_stock(self, session, symbol: str, quantity: int, price: float):
        """
        Attempt to sell `quantity` shares of `symbol` at `price`.
        Ensures the trader actually holds at least `quantity` shares of that symbol.
        Raises ValueError if not enough shares.
        Adds proceeds to the trader's balance.
        """
        # Check holdings
        holdings = self.get_holdings(session)
        current_holding = holdings.get(symbol, 0)
        if current_holding < quantity:
            raise ValueError(f"Not enough shares of {symbol} to sell.")

        # Increase balance
        proceeds = price * quantity
        self.balance += proceeds
        
        # Create a transaction record
        tx = Transaction(
            trader=self,
            symbol=symbol,
            quantity=quantity,
            price=price,
            transaction_type=TransactionType.SELL
        )
        session.add(tx)
        session.commit()

    def get_holdings(self, session) -> Dict[str, int]:
        """
        Return the current holdings of the trader as a dictionary:
            { 'SYMBOL': quantity, ... }
        """
        # Aggregate all BUY - SELL
        buys = (
            session.query(Transaction.symbol, Transaction.quantity)
            .filter_by(trader_id=self.id, transaction_type=TransactionType.BUY)
            .all()
        )
        sells = (
            session.query(Transaction.symbol, Transaction.quantity)
            .filter_by(trader_id=self.id, transaction_type=TransactionType.SELL)
            .all()
        )

        holdings = {}
        for symbol, qty in buys:
            holdings[symbol] = holdings.get(symbol, 0) + qty
        for symbol, qty in sells:
            holdings[symbol] = holdings.get(symbol, 0) - qty

        return {s: q for s, q in holdings.items() if q > 0}

    def get_transaction_history(self):
        """Return a list of all transactions (for convenience)."""
        return self.transactions

    def get_realized_profit(self, session) -> float:
        """
        Naive realized profit calculation:
        Sum of (sell proceeds) - (buy costs) for matching sold quantity.
        This is an *extremely* simplified version (no partial-lot handling).
        """
        # Sum total buy cost
        total_buy = (
            session.query(Transaction)
            .filter_by(trader_id=self.id, transaction_type=TransactionType.BUY)
        )
        total_buy_amount = sum(tx.price * tx.quantity for tx in total_buy)

        # Sum total sell proceeds
        total_sell = (
            session.query(Transaction)
            .filter_by(trader_id=self.id, transaction_type=TransactionType.SELL)
        )
        total_sell_amount = sum(tx.price * tx.quantity for tx in total_sell)

        return total_sell_amount - total_buy_amount
    
    def get_realized_profit_by_stock(self, session) -> Dict[str, float]:
        """
        Return the realized profit by stock symbol.
        """
        realized_profit_by_stock = {}
        for tx in self.transactions:
            if tx.transaction_type == TransactionType.SELL:
                realized_profit_by_stock[tx.symbol] = realized_profit_by_stock.get(tx.symbol, 0.0) + tx.price * tx.quantity
            elif tx.transaction_type == TransactionType.BUY:
                realized_profit_by_stock[tx.symbol] = realized_profit_by_stock.get(tx.symbol, 0.0) - tx.price * tx.quantity
        return realized_profit_by_stock
        

    def get_performance_stats(self, session, stocks_current_value) -> Dict[str, Any]:
        """
        Return some basic performance stats, e.g.:
        - current balance
        - total realized profit by summing all sell proceeds - buy costs
        - total real-time portfolio value by summing all current stock values
        - total equity by adding balance and portfolio value
        - current holdings
        - total realized profit by each stock
        """
        holdings = self.get_holdings(session)
        
        portfolio_value = 0.0
        for symbol, qty in holdings.items():
            # Use the current value of the stock
            stock_current_value = stocks_current_value.get(symbol)
            
            if stocks_current_value is None:
                raise ValueError(f"Stock {symbol} not found in current value data.")
            else:
                portfolio_value += stock_current_value * qty

        stats = {
            "current_balance": self.balance,
            "realized_profit": self.get_realized_profit(session),
            "portfolio_value": portfolio_value,
            "total_equity": self.balance + portfolio_value,
            "holdings": holdings,
            "realized_profit_by_stock": self.get_realized_profit_by_stock(session)
        }
        return stats


class Transaction(Base):
    """
    A Transaction ORM model to record buy/sell events, with a relationship back to the Trader.
    """
    __tablename__ = "transactions"

    id = Column(Integer, primary_key=True, autoincrement=True)
    trader_id = Column(Integer, ForeignKey("traders.id"), nullable=False)
    symbol = Column(String, nullable=False)
    quantity = Column(Integer, nullable=False)
    price = Column(Float, nullable=False)
    transaction_type = Column(Enum(TransactionType), nullable=False)
    timestamp = Column(DateTime, default=datetime.utcnow, nullable=False)

    trader = relationship("Trader", back_populates="transactions")


def get_scoped_session(db_url: str = "sqlite:///:memory:"):
    """
    Return a thread-safe scoped session connected to the given database URL.
    By default, uses an in-memory SQLite DB for demonstration.
    """
    engine = create_engine(db_url, echo=False)
    Base.metadata.create_all(engine)
    factory = sessionmaker(bind=engine)
    return scoped_session(factory)


In [17]:
def main():
    # Create a new database session (in-memory by default)
    session = get_scoped_session("sqlite:///db.sqlite")

    # Create a new trader with an initial balance if not already present
    if not session.query(Trader).first():
      trader = Trader(name="Alice", balance=100_000)
      session.add(trader)
      session.commit()
    else:
      trader = session.query(Trader).first()

    # Demonstrate some trades
    try:
        # Buy 10 shares of AAPL at $150
        trader.buy_stock(session, symbol="AAPL", quantity=2, price=150.0)
        # Buy 5 shares of TSLA at $700
        trader.buy_stock(session, symbol="TSLA", quantity=5, price=700.0)
        # Sell 3 shares of AAPL at $155
        trader.sell_stock(session, symbol="AAPL", quantity=7, price=155.0)
        # Sell 5 shares of TSLA at $710
        trader.sell_stock(session, symbol="TSLA", quantity=4, price=710.0)
    except ValueError as e:
        # Handle any errors (e.g., insufficient balance or shares)
        print("Trade Error:", e)

    # Print final holdings and performance statistics
    print("Final Holdings:", trader.get_holdings(session))
    print("Transaction History:")
    for tx in trader.get_transaction_history():
        print(f"  - {tx.transaction_type.value} {tx.quantity} of {tx.symbol} at ${tx.price} on {tx.timestamp}")

    stats = trader.get_performance_stats(session,
                                         
                                         stocks_current_value={"AAPL": 150.0, "TSLA": 680.0})
    print("\nPerformance Stats:")
    print(f"  Current Balance: ${stats['current_balance']:.2f}")
    print(f"  Realized Profit: ${stats['realized_profit']:.2f}")
    print(f"  Portfolio Value: ${stats['portfolio_value']:.2f}")
    print(f"  Total Equity:    ${stats['total_equity']:.2f}")
    print(f"  Holdings:        {stats['holdings']}")
    print(f"  Realized Profit by Stock: {stats['realized_profit_by_stock']}")


if __name__ == "__main__":
    main()


Final Holdings: {'AAPL': 30, 'TSLA': 1}
Transaction History:
  - BUY 10 of AAPL at $150.0 on 2025-01-01 18:10:59.883404
  - BUY 5 of TSLA at $700.0 on 2025-01-01 18:10:59.886387
  - SELL 3 of AAPL at $155.0 on 2025-01-01 18:10:59.890385
  - SELL 5 of TSLA at $710.0 on 2025-01-01 18:10:59.893386
  - BUY 10 of AAPL at $150.0 on 2025-01-01 18:12:55.265048
  - BUY 5 of TSLA at $700.0 on 2025-01-01 18:12:55.269042
  - SELL 3 of AAPL at $155.0 on 2025-01-01 18:12:55.277529
  - SELL 5 of TSLA at $710.0 on 2025-01-01 18:12:55.281530
  - BUY 10 of AAPL at $150.0 on 2025-01-01 18:13:00.061607
  - BUY 5 of TSLA at $700.0 on 2025-01-01 18:13:00.065611
  - SELL 3 of AAPL at $155.0 on 2025-01-01 18:13:00.070618
  - SELL 5 of TSLA at $710.0 on 2025-01-01 18:13:00.075610
  - BUY 10 of AAPL at $150.0 on 2025-01-01 18:30:20.520611
  - BUY 5 of TSLA at $700.0 on 2025-01-01 18:30:20.526134
  - SELL 3 of AAPL at $155.0 on 2025-01-01 18:30:20.537275
  - SELL 5 of TSLA at $710.0 on 2025-01-01 18:30:20.544271